## 문서 군집화 개념
- 문서 군집화(Document Clustering)는 비슷한 텍스트 구성의 문서를 군집화(`Clustering`)하는 것입니다. 
- 문서 군집화는 동일한 군집에 속하는 문서를 같은 카테고리 소속으로 분류할 수 있으므로 앞에서 소개한 텍스트 분류 기반의 문서 분류와 유사하다. 
- 데이터는 아래 링크를 통해서 다운로드 받는다. (Data Folder Link 클릭)
  + https://archive.ics.uci.edu/ml/datasets/Opinosis+Opinion+%26frasl%3B+Review


### 코랩 + 구글 드라이브 연동
- 우선 파일을 다운로드 받은 후, 구글 드라이브에 올려 놓는다. 

In [49]:
from google.colab import drive # 패키지 불러오기 
from os.path import join  

ROOT = "/content/drive"     # 드라이브 기본 경로
print(ROOT)                 # print content of ROOT (Optional)
drive.mount(ROOT)           # 드라이브 기본 경로 Mount

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
MY_GOOGLE_DRIVE_PATH = 'My Drive/Colab Notebooks/NLP/' # 프로젝트 경로
PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH) # 프로젝트 경로
print(PROJECT_PATH)

/content/drive/My Drive/Colab Notebooks/NLP/


In [51]:
%cd "{PROJECT_PATH}"

/content/drive/My Drive/Colab Notebooks/NLP


## 데이터 불러오기
- 데이터 불러오는 코드를 진행합니다. 
- 기존 코드에서 약간의 코드를 수정하였습니다.
- 기존 코드는 아래와 같습니다. 

```python
for file_ in all_files:
    # 개별 파일을 읽어서 DataFrame으로 생성 
    df = pd.read_table(file_,index_col=None, header=0,encoding='latin1')
    
    # 절대경로로 주어진 file 명을 가공. 만일 Linux에서 수행시에는 아래 \\를 / 변경. 맨 마지막 .data 확장자도 제거
    filename_ = file_.split('\\')[-1]
    filename = filename_.split('.')[0]

    #파일명 리스트와 파일내용 리스트에 파일명과 파일 내용을 추가. 
    filename_list.append(filename)
    opinion_text.append(df.to_string())
```

- 제가 수정한 코드는 아래와 같습니다. 

```python
for file_ in all_files:
  # 개별 파일을 읽어서 데이터 프레임으로 생성
  df = pd.read_table(file_, index_col=None, header=None, encoding='latin1')
  # print(df.head())
  df.columns = ['text']
  # print(df.head())

  # 절대 경로로 주어진 파일명을 가공. 리눅스에서 수행 시 \\를 /로 변경
  # 맨 마지막 .data 확장자도 제거
  filename_ = file_.split('/')[-1] # 만일 Linux에서 수행시에는 아래 \\를 / 변경. 맨 마지막 .data 확장자도 제거
  filename = filename_.split('.')[0]

  # 파일명 list와 파일 내용 list에 파일명과 파일 내용을 추가
  filename_list.append(filename)
  text_data = df.to_string()
  opinion_text.append(df.text.to_string(index=None))
```

In [82]:
import pandas as pd
import glob, os

# 폴더 경로(디렉터리) 설정
path = r'data/OpinosisDataset1.0/topics'

all_files = glob.glob(os.path.join(path, "*.data"))
print("전체 파일 개수:", len(all_files))
print(all_files[:2])

filename_list = []
opinion_text = []

# 개별 파일의 파일명은 filename_list
# 개별 파일의 파일 내용은 `DataFrame 로딩 후 다시 string으로 변환해 `opinion_text list`로 취합한다. 

for file_ in all_files:
  # 개별 파일을 읽어서 데이터 프레임으로 생성
  df = pd.read_table(file_, index_col=None, header=None, encoding='latin1')
  # print(df.head())
  df.columns = ['text']
  # print(df.head())

  # 절대 경로로 주어진 파일명을 가공. 리눅스에서 수행 시 \\를 /로 변경
  # 맨 마지막 .data 확장자도 제거
  filename_ = file_.split('/')[-1] # 만일 Linux에서 수행시에는 아래 \\를 / 변경. 맨 마지막 .data 확장자도 제거
  filename = filename_.split('.')[0]

  # 파일명 list와 파일 내용 list에 파일명과 파일 내용을 추가
  filename_list.append(filename)
  text_data = df.to_string()
  opinion_text.append(df.text.to_string(index=None))

# 파일명 list
document_df = pd.DataFrame({'filename': filename_list, 'opinion_text': opinion_text})
document_df.head()

전체 파일 개수: 51
['data/OpinosisDataset1.0/topics/battery-life_ipod_nano_8gb.txt.data', 'data/OpinosisDataset1.0/topics/screen_ipod_nano_8gb.txt.data']


,filename,opinion_text
0,battery-life_ipod_nano_8gb,short battery life I moved up from an 8...
1,screen_ipod_nano_8gb,"As always, the video screen is sharp and bri..."
2,features_windows7,"I had to uninstall anti, virus and selected ..."
3,sound_ipod_nano_8gb,headphone jack i got a clear case for it and...
4,screen_garmin_nuvi_255W_gps,It is easy to read and when touching the sc...


## TF-IDF 피처 벡터화된 행렬
- 피처 벡터화된 행렬을 구한다. 
- 각 파일 이름(filename) 자체만으로 의견(opinion)의 텍스트(text)가 어떠한 제품/서비스에 대한 리뷰인지 잘 알 수 있다. 
- 이제 TF-IDF에 대해 피처 벡터화된 행렬을 구하도록 한다. 

In [83]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

nltk.download('punkt')
nltk.download('wordnet')

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [84]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words="english", ngram_range=(1, 2), min_df=0.05, max_df=0.85)

# opinion_text
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


## K-Means 모형
- TF-IDF 변환된 피처 벡터화 행렬 데이터에 대해 군집화를 수행하여 어떤 문서끼리 군집되는지 확인해본다. 
- 군집화 기법은 K-평균을 적용한다. 
  + 5개의 중심으로 어떻게 군집화되는지 확인해본다. 
- K-Means 알고리즘에 관한 설명이 필요하신 분은 다음 셀에서 확인하도록 한다. 

In [100]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/4b5d3muPQmA" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

In [85]:
from sklearn.cluster import KMeans

# 5개의 집합으로 군집화 수행
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)

cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

- 각 데이터별로 할당된 군집의 레이블을 파일명과 파일 내용을 가지고 있는 `document_df`에 `cluster_label` 칼럼을 추가해 저장한다. 


## 모형 결과 확인
- 각각의 모형에 대한 결과를 확인하도록 한다. 

In [86]:
document_df['cluster_label'] = cluster_label
document_df.head(20)

,filename,opinion_text,cluster_label
0,battery-life_ipod_nano_8gb,short battery life I moved up from an 8...,4
1,screen_ipod_nano_8gb,"As always, the video screen is sharp and bri...",4
2,features_windows7,"I had to uninstall anti, virus and selected ...",4
3,sound_ipod_nano_8gb,headphone jack i got a clear case for it and...,4
4,screen_garmin_nuvi_255W_gps,It is easy to read and when touching the sc...,4
5,video_ipod_nano_8gb,"I bought the 8, gig Ipod Nano that has the bu...",4
6,updates_garmin_nuvi_255W_gps,Another thing to consider was that I paid $50...,3
7,speed_garmin_nuvi_255W_gps,Another feature on the 255w is a display of t...,4
8,screen_netbook_1005ha,Keep in mind that once you get in a room ful...,4
9,directions_garmin_nuvi_255W_gps,You also get upscale features like spoken di...,3


- 각 데이터별로 할당된 군집의 레이블을 파일명과 파일 내용을 가지고 있는 document_df DataFrame에 `cluster_label` 컬람을 추가해 저장합니다. 
- 각 파일명은 의견 리뷰에 대한 주제를 나타냅니다. 
- 판다스 DataFrame의 `sort_values(by=칼럼명)`을 수행하면 인자로 입력된 `칼럼명`으로 데이터를 정렬할 수 있다.

In [88]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
45,comfort_honda_accord_2008,"Drivers seat not comfortable, the car itself...",0
27,comfort_toyota_camry_2007,Ride seems comfortable and gas mileage fairl...,0
36,gas_mileage_toyota_camry_2007,Ride seems comfortable and gas mileage fairl...,0
37,interior_honda_accord_2008,I love the new body style and the interior i...,0
24,interior_toyota_camry_2007,"First of all, the interior has way too many ...",0
44,mileage_honda_accord_2008,"It's quiet, get good gas mileage and looks c...",0
38,performance_honda_accord_2008,"Very happy with my 08 Accord, performance is ...",0
50,quality_toyota_camry_2007,I previously owned a Toyota 4Runner which ha...,0
43,seats_honda_accord_2008,Front seats are very uncomfortab...,0
47,transmission_toyota_camry_2007,"After slowing down, transmission has to be ...",0


- `cluster #0`는 토요타와 혼다 등 자동차로 분류가 된 것으로 확인할 수 있다. 

In [89]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
22,food_holiday_inn_london,The room was packed to capacity with queues ...,1
23,food_swissotel_chicago,The food for our event was delicio...,1
42,free_bestwestern_hotel_sfo,The wine reception is a great idea as it is n...,1
48,location_bestwestern_hotel_sfo,"Good Value good location , ideal choi...",1
26,location_holiday_inn_london,Great location for tube and we crammed in a ...,1
31,price_holiday_inn_london,"All in all, a normal chain hotel on a nice lo...",1
41,service_bestwestern_hotel_sfo,Both of us having worked in tourism for over...,1
25,service_holiday_inn_london,"not customer, oriented hotelvery low service ...",1
29,service_swissotel_hotel_chicago,Mediocre room and service for a very extrava...,1
40,staff_bestwestern_hotel_sfo,Staff are friendly and helpful...,1


- 이번에는 호텔에 관한 리뷰로 되어 있는 것을 확인할 수 있다. 

In [90]:
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
39,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and v...",2
46,parking_bestwestern_hotel_sfo,Parking was expensive but I think this is co...,2
33,room_holiday_inn_london,"We arrived at 23,30 hours and they could not ...",2
49,rooms_bestwestern_hotel_sfo,"Great Location , Nice Rooms , Helpless Co...",2
20,rooms_swissotel_chicago,The Swissotel is one of our favorite hotels i...,2


- 이번에도 마찬가지로 호텔과 관련된 리뷰로 확인되고 있습니다.

In [91]:
document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
14,accuracy_garmin_nuvi_255W_gps,", and is very, very accura...",3
19,buttons_amazon_kindle,I thought it would be fitting to christen my ...,3
9,directions_garmin_nuvi_255W_gps,You also get upscale features like spoken di...,3
15,display_garmin_nuvi_255W_gps,3 quot widescreen display was a bon...,3
13,satellite_garmin_nuvi_255W_gps,It's fast to acquire satellit...,3
16,speed_windows7,"Windows 7 is quite simply faster, more stable...",3
6,updates_garmin_nuvi_255W_gps,Another thing to consider was that I paid $50...,3
12,voice_garmin_nuvi_255W_gps,The voice prompts and maps are wonderful es...,3


- 주로 차량용 네비게이션으로 군집이 구성되어 있음을 확인할 수 있다. 


In [92]:
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
35,battery-life_amazon_kindle,After I plugged it in to my USB hub on my co...,4
0,battery-life_ipod_nano_8gb,short battery life I moved up from an 8...,4
10,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 23Wh...",4
28,eyesight-issues_amazon_kindle,It feels as easy to read as the K1 but doesn...,4
2,features_windows7,"I had to uninstall anti, virus and selected ...",4
21,fonts_amazon_kindle,Being able to change the font sizes is aweso...,4
17,keyboard_netbook_1005ha,", I think the new keyboard rivals the great ...",4
34,navigation_amazon_kindle,"In fact, the entire navigation structure has...",4
30,performance_netbook_1005ha,The Eee Super Hybrid Engine utility lets use...,4
18,price_amazon_kindle,"If a case was included, as with the Kindle 1...",4


- 킨들, 아이팟, 넷북 등 주로 포터블 전자기기에 대한 리뷰로 군집화된 것을 확인할 수 있다. 


## K-Means 모형 업데이트
- 이번에는 군집을 3개로 좁혀서 실행하도록 합니다. 기존 코드와 큰 차이점은 없습니다. 

In [95]:
from sklearn.cluster import KMeans

# 5개의 집합으로 군집화 수행
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)

cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

document_df['cluster_label'] = cluster_label

In [96]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
45,comfort_honda_accord_2008,"Drivers seat not comfortable, the car itself...",0
27,comfort_toyota_camry_2007,Ride seems comfortable and gas mileage fairl...,0
36,gas_mileage_toyota_camry_2007,Ride seems comfortable and gas mileage fairl...,0
37,interior_honda_accord_2008,I love the new body style and the interior i...,0
24,interior_toyota_camry_2007,"First of all, the interior has way too many ...",0
44,mileage_honda_accord_2008,"It's quiet, get good gas mileage and looks c...",0
38,performance_honda_accord_2008,"Very happy with my 08 Accord, performance is ...",0
50,quality_toyota_camry_2007,I previously owned a Toyota 4Runner which ha...,0
43,seats_honda_accord_2008,Front seats are very uncomfortab...,0
47,transmission_toyota_camry_2007,"After slowing down, transmission has to be ...",0


In [97]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
14,accuracy_garmin_nuvi_255W_gps,", and is very, very accura...",1
35,battery-life_amazon_kindle,After I plugged it in to my USB hub on my co...,1
0,battery-life_ipod_nano_8gb,short battery life I moved up from an 8...,1
10,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 23Wh...",1
19,buttons_amazon_kindle,I thought it would be fitting to christen my ...,1
9,directions_garmin_nuvi_255W_gps,You also get upscale features like spoken di...,1
15,display_garmin_nuvi_255W_gps,3 quot widescreen display was a bon...,1
28,eyesight-issues_amazon_kindle,It feels as easy to read as the K1 but doesn...,1
2,features_windows7,"I had to uninstall anti, virus and selected ...",1
21,fonts_amazon_kindle,Being able to change the font sizes is aweso...,1


In [98]:
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
39,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and v...",2
22,food_holiday_inn_london,The room was packed to capacity with queues ...,2
23,food_swissotel_chicago,The food for our event was delicio...,2
42,free_bestwestern_hotel_sfo,The wine reception is a great idea as it is n...,2
48,location_bestwestern_hotel_sfo,"Good Value good location , ideal choi...",2
26,location_holiday_inn_london,Great location for tube and we crammed in a ...,2
46,parking_bestwestern_hotel_sfo,Parking was expensive but I think this is co...,2
31,price_holiday_inn_london,"All in all, a normal chain hotel on a nice lo...",2
33,room_holiday_inn_london,"We arrived at 23,30 hours and they could not ...",2
49,rooms_bestwestern_hotel_sfo,"Great Location , Nice Rooms , Helpless Co...",2


- 위 3개의 군집별로 겹치지 않게 잘 리뷰된 것을 확인할 수 있다. 

## 군집별 핵심 단어 추출하기
- 각 군집(`Cluster`)에 속한 문서들 중 핵심 단어들을 추출하도록 한다. 
- KMeans 객체는 각 군집을 구성하는 단어 피처가 군집의 중심(`Centroid`)를 기준으로 얼마나 가깝게 위치해 있는 `clusters_centers_`라는 속성으로 제공한다. 


In [101]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape :', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape : (3, 2139)
[[0.         0.         0.00127568 ... 0.         0.         0.        ]
 [0.01849092 0.01849092 0.00214253 ... 0.         0.         0.00474981]
 [0.         0.         0.0069575  ... 0.00327434 0.00350587 0.        ]]


- 위 결과값을 보면, (3, 2409) 배열이다. 
- 이 뜻은 군집이 3개, word 피처가 2409개로 구성되어 있음을 의미한다. 
- get_cluster_details() 함수를 구성하도록 한다. 

In [102]:
# 군집별 top n 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명들을 반환함. 
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}
    
    # cluster_centers array 의 값이 큰 순으로 정렬된 index 값을 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.  
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    #개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화. 
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        # cluster_centers_.argsort()[:,::-1] 로 구한 index 를 이용하여 top n 피처 단어를 구함. 
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]
        
        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함 
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        
        # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값, 그리고 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames
        
    return cluster_details

- 먼저, `ndarray`의 `argsort()[:,::-1]`를 이용하면 cluster_centers 배열 내 값이 큰 순으로 정렬된 위치 인덱스 값을 반환한다.
  + 큰 값을 가진 배열 내 위치 인덱스 값을 반환하는 것임을 유의한다. 
- 이 위치 인덱스 값이 필요한 이유는 핵심 단어 피처의 이름을 출력하기 위한 것이다.  
- 이 위치 인덱스 값이 필요한 이유는 핵심 단어 피처의 이름을 출력하기 위한 것이다. 
- `cluster_centers_` 배열 내에서 가장 값이 큰 데이터의 위치 인덱스를 추출한 뒤 해당 인덱스를 이용해 핵심 단어 이름과 그 때의 상대 위치 값을 추출해 `cluster_details`라는 `Dict` 객체 변수에 기록하고 반환하는 것이 `get_cluster_details()` 함수의 주요 로직이다. 
- 이번에는 개별 군집번호, 핵심 단어, 핵심 단어 중심 위치 상대값, 파일명 속성 값 정보를 출력하도록 해본다.  

In [106]:
def print_cluster_details(cluster_details):
  for cluster_num, cluster_detail in cluster_details.items():
    print('### Cluster {0}'.format(cluster_num))
    print('Top features:', cluster_detail['top_features'])
    print('Reveiws 파일명 :', cluster_detail['filenames'][:7])
    print('==========================')

- 이제 위에서 생성한 함수를 적용해보독 합니다. 

In [107]:
feature_names = tfidf_vect.get_feature_names()
cluster_details = get_cluster_details(cluster_model=km_cluster, 
                                      cluster_data=document_df, 
                                      feature_names = feature_names, 
                                      clusters_num = 3, 
                                      top_n_features=10)

print_cluster_details(cluster_details)

### Cluster 0
Top features: ['interior', 'seat', 'mileage', 'comfortable', 'car', 'gas', 'transmission', 'gas mileage', 'ride', 'comfort']
Reveiws 파일명 : ['interior_toyota_camry_2007', 'comfort_toyota_camry_2007', 'gas_mileage_toyota_camry_2007', 'interior_honda_accord_2008', 'performance_honda_accord_2008', 'seats_honda_accord_2008', 'mileage_honda_accord_2008']
### Cluster 1
Top features: ['screen', 'battery', 'life', 'battery life', 'keyboard', 'kindle', 'size', 'easy', 'button', 'voice']
Reveiws 파일명 : ['battery-life_ipod_nano_8gb', 'screen_ipod_nano_8gb', 'features_windows7', 'sound_ipod_nano_8gb', 'screen_garmin_nuvi_255W_gps', 'video_ipod_nano_8gb', 'updates_garmin_nuvi_255W_gps']
### Cluster 2
Top features: ['room', 'hotel', 'service', 'location', 'staff', 'food', 'clean', 'bathroom', 'parking', 'room wa']
Reveiws 파일명 : ['rooms_swissotel_chicago', 'food_holiday_inn_london', 'food_swissotel_chicago', 'service_holiday_inn_london', 'location_holiday_inn_london', 'service_swissotel_h

- 위 결과에 대한 해석은 각자에게 맡기도록 한다. 